## Índice de marginación por entidad federativa y municipios. 2015 y 2020.

In [10]:
import os
import pandas as pd
import urllib.request as ur

from datetime import datetime
from pathlib import Path

In [2]:
#!pip install openpyxl
#!pip install --upgrade pandas
# print(pd.__version__)
#!pip install --upgrade xlrd
#!pip install pyarrow

# Extraccion de datos

In [11]:
dataPath = os.getcwd() + "/data/"
urlMunicipios2015 = "http://www.conapo.gob.mx/work/models/CONAPO/Marginacion/Datos_Abiertos/Municipio/IMM_DP2_2015.xlsx"
urlEntidades2015 = "http://www.conapo.gob.mx/work/models/CONAPO/Marginacion/Datos_Abiertos/Entidad_Federativa/IME_DP2_2015.xlsx"
urlMunicipios2020 = "http://www.conapo.gob.mx/work/models/CONAPO/Marginacion/Datos_Abiertos/Municipio/IMM_2020.xls"
urlEntidades2020 = "http://www.conapo.gob.mx/work/models/CONAPO/Marginacion/Datos_Abiertos/Entidad_Federativa/IME_2020.xls"
urls = (urlMunicipios2015, urlEntidades2015, urlMunicipios2020, urlEntidades2020)

In [15]:
if not os.path.exists(dataPath):
    os.mkdir(dataPath)
    ur.urlretrieve(urlMunicipios2015, dataPath + "IMM_2015.xlsx")
    ur.urlretrieve(urlMunicipios2020, dataPath + "IMM_2020.xlsx")
    ur.urlretrieve(urlEntidades2015, dataPath + "IME_2015.xlsx")
    ur.urlretrieve(urlEntidades2020, dataPath + "IME_2020.xlsx")    
else:
    if not os.path.exists(dataPath + "IMM_2015.xlsx"): #con que uno no exista
        ur.urlretrieve(urlMunicipios2015, dataPath + "IMM_2015.xlsx")
        ur.urlretrieve(urlMunicipios2020, dataPath + "IMM_2020.xlsx")
        ur.urlretrieve(urlEntidades2015, dataPath + "IME_2015.xlsx")
        ur.urlretrieve(urlEntidades2020, dataPath + "IME_2020.xlsx")

# Creamos  archivo .txt para dar detalles de la descarga
with open(dataPath + "info_05_marginalidad.txt", 'w') as f:

    lista_archivos = [
        "IMM_2015.xlsx", "IMM_2020.xlsx", "IME_2015.xlsx", "IME_2020.xlsx"
    ]

    fecha_hoy = datetime.now()
    fecha_hoy_str = fecha_hoy.strftime("%Y-%m-%d %H:%M:%S")

    f.write("Índice de marginación por entidad federativa y municipios. México - 2015 y 2020.\n")
    info = """
Se presentan los resultados del índice de marginación por entidad federativa y municipio con base en el censo de 2015 y 2020.
"""
    f.write(info + '\n')
    f.write(f"Descargado el {fecha_hoy_str} \n")
    f.write(f"Desde: {urls}\n")
    f.write("Archivos descargados:\n")
    for i, a in enumerate(lista_archivos, start=1):
        f.write(f"\t{i}. {a}")

# Transformación de datos 

## Municipales

In [ ]:
IMM_2015 = pd.read_excel(dataPath + "IMM_2015.xlsx", sheet_name = 0, skiprows = 5, usecols = [0,1, 2, 3, 14])
IMM_2020 = pd.read_excel(dataPath + "IMM_2020.xlsx", sheet_name = 0, skiprows = 5, usecols = [0, 1, 2, 3, 14])

In [ ]:
McolNames = {"CVE_ENT":"clave_entidad",
             "NOM_ENT":"nombre_entidad", 
             "CVE_MUN":"clave_municipio",
             "NOM_MUN":"nombre_municipio",
             "IM_2015":"inidice_marginalizacion_2015"}

IMM_2015.rename(columns = McolNames, inplace = True)
IMM_2015.drop([0,1, 2459, 2460],axis = 0, inplace = True)

In [ ]:
McolNames = {"CVE_ENT":"clave_entidad",
             "NOM_ENT":"nombre_entidad", 
             "CVE_MUN":"clave_municipio",
             "NOM_MUN":"nombre_municipio",
             "IM_2020":"inidice_marginalizacion_2020"}

IMM_2020.rename(columns = McolNames, inplace = True)
IMM_2020.drop([0,1, 2471, 2472],axis = 0, inplace = True)

In [ ]:
IMM = pd.merge(IMM_2020, 
               IMM_2015[["clave_municipio", "inidice_marginalizacion_2015"]],
              on = "clave_municipio",
              how = "outer")
#cambio orden de columnas
IMM = IMM[["clave_entidad", 
           "nombre_entidad", 
           "clave_municipio", 
           "nombre_municipio", 
           "inidice_marginalizacion_2015", 
           "inidice_marginalizacion_2020"]]

#correcion tipo y formato de columna clave_municipio
IMM["clave_municipio"] = IMM["clave_municipio"].astype("str")
IMM["clave_municipio"] = IMM["clave_municipio"].str[:-2]
IMM["clave_municipio"] = IMM["clave_municipio"].str.rjust(5, "0")

#orden nnatural
IMM = IMM.sort_values(["clave_entidad", "clave_municipio"])

#cambio de object a float de indice de marginacion
IMM[["inidice_marginalizacion_2015", "inidice_marginalizacion_2020"]] = IMM[["inidice_marginalizacion_2015", "inidice_marginalizacion_2020"]].astype(float)

In [ ]:
#Hay 12 nulos, deben ser nuevos municipios
IMM.isnull().sum()

clave_entidad                    0
nombre_entidad                   0
clave_municipio                  0
nombre_municipio                 0
inidice_marginalizacion_2015    12
inidice_marginalizacion_2020     0
dtype: int64

In [ ]:
IMM

,clave_entidad,nombre_entidad,clave_municipio,nombre_municipio,inidice_marginalizacion_2015,inidice_marginalizacion_2020
0,01,Aguascalientes,01001,Aguascalientes,51.309665,60.318795
1,01,Aguascalientes,01002,Asientos,47.064347,56.546071
2,01,Aguascalientes,01003,Calvillo,47.943584,57.058251
3,01,Aguascalientes,01004,Cosío,47.651559,57.114030
4,01,Aguascalientes,01005,Jesús María,49.701963,59.011762
...,...,...,...,...,...,...
2464,32,Zacatecas,32054,Villa Hidalgo,45.558073,55.044146
2465,32,Zacatecas,32055,Villanueva,47.850941,56.747058
2466,32,Zacatecas,32056,Zacatecas,51.667211,60.176191
2467,32,Zacatecas,32057,Trancoso,48.270299,56.958073


## Estatales

In [ ]:
IME_2015 = pd.read_excel(dataPath + "IME_2015.xlsx", sheet_name = 0, skiprows = 4, usecols = [0,1, 12])
IME_2020 = pd.read_excel(dataPath + "IME_2020.xlsx", sheet_name = 0, skiprows = 4, usecols = [0,1, 12])

In [ ]:
EcolNames = {"CVE_ENT":"clave_entidad",
            "NOM_ENT":"nombre_entidad",
            "IM_2015":"inidice_marginalizacion_2015"}
IME_2015.rename(columns = EcolNames, inplace = True)
IME_2015.drop([0,1, 34, 35], axis = 0, inplace = True)

In [ ]:
EcolNames = {"CVE_ENT":"clave_entidad",
            "NOM_ENT":"nombre_entidad",
            "IM_2020":"inidice_marginalizacion_2020"}
IME_2020.rename(columns = EcolNames, inplace = True)
IME_2020.drop([0,1, 34, 35], axis = 0, inplace = True)

In [ ]:
#merge 2015 y 2020
IME = pd.merge(IME_2015[["clave_entidad", "inidice_marginalizacion_2015"]], 
               IME_2020, 
               on = "clave_entidad",
               how = "outer")
#cambio orden de columnas
IME = IME[["clave_entidad",
          "nombre_entidad",
          "inidice_marginalizacion_2015",
          "inidice_marginalizacion_2020"]]
#cambio object a float indices de marginacion
IME[["inidice_marginalizacion_2015","inidice_marginalizacion_2020"]] = IME[["inidice_marginalizacion_2015","inidice_marginalizacion_2020"]].astype("float")
IME

,clave_entidad,nombre_entidad,inidice_marginalizacion_2015,inidice_marginalizacion_2020
0,01,Aguascalientes,17.883964,22.205721
1,02,Baja California,18.464138,21.380297
2,03,Baja California Sur,17.320595,21.473388
3,04,Campeche,13.684555,17.805051
4,05,Coahuila de Zaragoza,18.400944,22.545684
5,06,Colima,17.432451,21.532329
6,07,Chiapas,8.449801,11.998654
7,08,Chihuahua,16.821709,20.015271
8,09,Ciudad de México,19.398145,23.143109
9,10,Durango,14.942236,18.472740


# Guardado de datos

In [ ]:
#pesa 94.5 kb
IMM.to_parquet(dataPath + "IndiceMarginacionMunicipal.parquet")
#pesa 181.4
#IMM.to_csv(dataPath + "IndiceMarginacionMunicipal.csv")

#pesa 1.7 kb
IME.to_parquet(dataPath + "IndiceMarginacionEstatal.parquet")
#pesa 5.7 kb
#IME.to_csv(dataPath + "IndiceMarginacionEstatal.csv")

